In [ ]:
import scanpy as sc #important note: the source code for scanpy heatmap was modified locally to suite the visual preferences (Ziadoon Al-Akashi)
import anndata as ad
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import os
from matplotlib.colors import Normalize
import gseapy as gp
from typing import Union, Optional, Tuple, Mapping
from matplotlib.colors import Normalize
import matplotlib.gridspec as gridspec
from matplotlib.patches import Patch
from matplotlib.lines import Line2D


# Reading and processing

In [ ]:
data_folder = 'E:/scRNA seq Data/'

In [ ]:
ribo_url = "http://software.broadinstitute.org/gsea/msigdb/download_geneset.jsp?geneSetName=KEGG_RIBOSOME&fileType=txt"
ribo_genes = pd.read_table(ribo_url, skiprows=2, header = None, names=['gene'])
ribo_genes

In [ ]:
all_adata = []

h5_paths = [
    data_folder + 'ACL_iNSCs_PN5',
    data_folder + 'ACL_iNSCs_PN21',
    data_folder + 'ACL_iNSCs_PN41',
    data_folder + 'ACL_iNSCs_PN59',
]

In [ ]:
def read_and_append_data(path, all_adata):
    adata = sc.read_10x_mtx(path, var_names="gene_symbols", cache=True)
    sample_name = path.split('_')[2] 
    adata.obs['Sample'] = sample_name
    all_adata.append(adata)

def process_data(all_adata, ribo_genes):
    for i, adata in enumerate(all_adata):
        print(f"Before filtering - Number of cells: {adata.n_obs}")

        sc.pp.filter_cells(adata, min_genes=400)
        adata.var['mt'] = adata.var_names.str.startswith('MT-')
        adata.var['ribo'] = adata.var_names.isin(ribo_genes['gene'])
        sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribo'], percent_top=None, log1p=False, inplace=True)
        
        upper_lim = np.quantile(adata.obs.n_genes_by_counts.values, 0.98)
        adata = adata[adata.obs.n_genes_by_counts < upper_lim]

        adata = adata[adata.obs.pct_counts_mt < 10, :].copy()
        all_adata[i] = adata  

        print(f"After filtering - Number of cells: {adata.n_obs}")

    return all_adata

In [ ]:
for path in h5_paths:
    read_and_append_data(path, all_adata)

In [ ]:
processed_data = process_data(all_adata, ribo_genes)

In [ ]:
all_adata[1]

In [ ]:
concatenated_processed_data = ad.concat(all_adata)

In [ ]:
concatenated_processed_data.obs_names_make_unique()

In [ ]:
concatenated_processed_data.obs.sort_values('n_genes_by_counts')

In [ ]:
sc.pl.violin(concatenated_processed_data, 
             ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', 'pct_counts_ribo'], 
             jitter=0.4, 
             multi_panel=True)

In [ ]:
import scrublet as scr

In [ ]:
adata_for_scrublet = concatenated_processed_data.copy()

In [ ]:
scrub = scr.Scrublet(adata_for_scrublet.X)

In [ ]:
doublet_scores, predicted_doublets = scrub.scrub_doublets()
plt.hist(doublet_scores, bins=50, color='blue', edgecolor='black')
plt.xlabel('Doublet Scores')
plt.ylabel('Frequency')
plt.title('Distribution of Doublet Scores')
plt.show()

In [ ]:
concatenated_processed_data.obs['doublet_scores'] = doublet_scores

In [ ]:
concatenated_processed_data = concatenated_processed_data[~predicted_doublets, :]

In [ ]:
adata = concatenated_processed_data.copy()

# Normalize and highy variable genes

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=2, min_disp=0.5)
adata.raw = adata
adata_raw = sc.AnnData(X=adata.raw.X, obs=adata.obs, var=adata.var, uns=adata.uns)
adata = adata[:, adata.var.highly_variable]

In [ ]:
sc.set_figure_params(transparent=True, fontsize=12)
custom_palette = sns.color_palette("hls", 4)
sns.set_palette('colorblind')

In [ ]:
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata, log=False, n_pcs=50, save=True)

In [ ]:
sc.pl.pca(adata, color='Sample', save='final_PCA_.png')

In [ ]:
adata.obs

In [ ]:
sc.pp.neighbors(adata, n_neighbors=16, n_pcs=20)

# Compute UMAP
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.8)

In [ ]:
file_path = "final_processed_adata.h5ad"

In [ ]:
#adata.write(file_path)

In [ ]:
adata = sc.read(file_path)

In [ ]:
color_s = ['#800000', '#9A6324', '#808000', '#469990', '#000075', '#e6194B', '#f58231', '#ffe119', '#bfef45', '#3cb44b', '#42d4f4', '#4363d8', '#911eb4', '#f032e6', '#a9a9a9', '#fabed4', '#ffd8b1']
legend_handles = [Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=f'{i}') for i, color in enumerate(color_s)]

fig, ax = plt.subplots(figsize=(1, 1))
ax.axis('off')
ax.legend(handles=legend_handles, ncol=3, loc='center')
plt.savefig('legend_clusters.png', bbox_inches='tight', transparent=True, dpi=300)
plt.show()

In [ ]:
# UMAP Leiden 
color_s = ['#800000', '#9A6324', '#808000', '#469990', '#000075','#e6194B', '#f58231', '#ffe119', '#bfef45', '#3cb44b',
          '#42d4f4', '#4363d8', '#911eb4', '#f032e6', '#a9a9a9','#fabed4', '#ffd8b1', '#aaffc3', '#dcbeff']
sc.set_figure_params(transparent=True, dpi=80, fontsize=14, dpi_save=300, figsize=(10, 8))
sns.set_palette("deep")
plt.figure(figsize=(10, 8))

sc.pl.umap(adata,
           color=['leiden'],
           palette=color_s,
           use_raw=False,
           title=[''],
           legend_loc=False, 
           legend_fontsize=8, 
           legend_fontoutline=2,  
           #frameon=False,
           size=30,  
           alpha=1, 
           linewidth=0.5,  
           show=False  
           )

plt.title('Leiden Clusters', fontsize=20)
plt.xlabel('UMAP1', fontsize=20)
plt.ylabel('UMAP2', fontsize=20)

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.savefig('final_umap_leiden_res8.png', bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
color_s1 = {'PN5': 'darkred', 'PN21': 'purple', 'PN41': 'darkorange', 'PN59': 'gold'}

In [ ]:
color_s1 = {'PN5': 'darkred', 'PN21': 'purple', 'PN41': 'darkorange', 'PN59': 'gold'}
legend_handles1 = [Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=label) for label, color in color_s1.items()]

fig, ax = plt.subplots(figsize=(1, 1))
ax.axis('off')
ax.legend(handles=legend_handles1, ncol=1, loc='center')

plt.savefig('legend_clusters_sample.png', bbox_inches='tight', transparent=True, dpi=300)
plt.show()

In [ ]:
# UMAP Samples

sc.set_figure_params(transparent=True, dpi=80, fontsize=14, dpi_save=300, figsize=(10, 8))
sns.set_palette("deep")

plt.figure(figsize=(10, 8))
sc.pl.umap(adata,
           color='Sample',  
           palette=color_s1,  
           use_raw=False,
           title=[''],
           legend_loc=False, 
           size=30,
           alpha=0.8,
           linewidth=0.5,
           show=False
           )

#plt.title('Samples', fontsize=20)
plt.xlabel('UMAP1', fontsize=20)
plt.ylabel('UMAP2', fontsize=20)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.savefig('final_umap_sample_res3.png', bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
# Sub UMAP Samples

sc.set_figure_params(transparent=True, dpi=80, fontsize=14, dpi_save=300)
sns.set_palette("deep")
fig, axs = plt.subplots(4, 1, figsize=(2, 5), sharex=True, sharey=True)
sample_groups = ['PN5', 'PN21', 'PN41', 'PN59']

for i, sample_group in enumerate(sample_groups):
    ax = axs[i]
    sc.pl.umap(adata, color=['Sample'], 
               groups=[sample_group], 
               title=[''],
               size=2,
               alpha=0.8,
               linewidth=0.1,
               legend_loc=None, 
               show=False,
               ax=ax)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('equal')

plt.subplots_adjust(hspace=0.05)
plt.savefig('combined_umap_plots_column_with_boxes.png', bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
sample_counts = adata.obs['Sample'].value_counts()
print(sample_counts)

In [ ]:
sample_counts = adata.obs['Sample'].value_counts()

sample_counts = sample_counts.sort_index(ascending=False)
plt.figure(figsize=(4, 4))
ax = plt.axes()
sample_counts.plot(kind='barh', color=[color_s1.get(sample, 'slategray') for sample in sample_counts.index], ax=ax)
ax.grid(False)  
plt.xlabel('Number of cells')
alpha=0.8

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#ax.spines['bottom'].set_visible(False)
#ax.spines['left'].set_visible(False)

for i, value in enumerate(sample_counts):
    plt.text(value + 0.05, i, str(value), ha='left', va='center')

plt.xlim(0, 5500)

plt.savefig('final_number_of_cells_in_sample.png', bbox_inches='tight', dpi=300,  transparent=True)
plt.show()


In [ ]:
genes_to_plot = ['HOXB9', 'HOXB4', 'HOXB2', 'LMX1B', 'TH', 'NEFL', 'SOX1', 'NES', 'SOX2', 'HES5']

num_genes = len(genes_to_plot)
num_cols = min(5, num_genes)
num_rows = -(-num_genes // num_cols)  
fig, axs = plt.subplots(num_rows, num_cols, figsize=(3 * num_cols, 3 * num_rows), sharex=True, sharey=True)
sc.set_figure_params(transparent=True, dpi=300, fontsize=20)

for gene, ax in zip(genes_to_plot, axs.flatten()):
    sc.pl.umap(adata,
               color=gene,
               cmap='rocket_r',
               frameon=False,
               vmin=0,
               size=5,
               use_raw=True,
               alpha=0.8,
               linewidth=0.8,
               legend_fontsize=1,
               show=False,
               ax=ax
               )
    ax.set_aspect('equal')
for ax in axs.flatten()[num_genes:]:
    ax.remove()

#plt.savefig('final_genes_on_clusters.png', bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
sc.tl.rank_genes_groups(adata, groupby='Sample',reference='PN59', method='wilcoxon', use_raw=True)

In [ ]:
sc.set_figure_params(dpi=300)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False, show=False, ncols=3)
plt.savefig('final_wilcoxon ranked genes vs. PN59.png', bbox_inches='tight', dpi=300,  transparent=True)

In [ ]:
top_n_genes = 25
marker_genes_df = pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(top_n_genes)
file_path = 'final_wilcoxon_sample_25.csv'

marker_genes_df.to_csv(file_path, index=False) 
marker_genes_df

In [ ]:
#subset = adata[adata.obs['Sample'].isin(['PN5', 'PN59'])].copy()
#sc.tl.rank_genes_groups(subset, groupby='Sample', method='t-test', use_raw=True)

In [ ]:
dedf = sc.get.rank_genes_groups_df(adata, group='PN5', key='rank_genes_groups')
upreg_df = dedf[(dedf.pvals < 0.05) & (dedf.logfoldchanges > 1.5)]
downreg_df = dedf[(dedf.pvals < 0.05) & (dedf.logfoldchanges < -1.5)]

upreg_df = upreg_df.sort_values('logfoldchanges', ascending=False).reset_index(drop=True)
downreg_df = downreg_df.sort_values('logfoldchanges', ascending=True).reset_index(drop=True)

upreg_genes = upreg_df.names.tolist()
downreg_genes = downreg_df.names.tolist()

upreg_genes10 = upreg_df[:25].names.tolist()
downreg_genes10 = downreg_df[:25].names.tolist()

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 10))

ax2 = ax1.twinx()
ax1.barh(range(len(upreg_genes10)), upreg_df[:25]['logfoldchanges'], color='darkgreen', label='Upregulated', alpha=0.7)
ax2.barh(range(len(downreg_genes10)), downreg_df[:25]['logfoldchanges'], color='darkred', label='Downregulated', alpha=0.7)

ax1.axvline(x=0, color='black', linestyle='-')  
ax1.set_yticks(range(len(downreg_genes10)))
ax1.set_yticklabels(downreg_genes10)
ax2.set_yticks(range(len(upreg_genes10)))
ax2.set_yticklabels(upreg_genes10)

ax1.set_xlabel('Log-Fold Change')
ax1.set_xlim([-30, 30])

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper right',  fontsize=14)

ax1.grid(False)
ax2.grid(False)

ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['left'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['left'].set_visible(False)

#plt.suptitle('', fontsize=20, y=0.95)
plt.title('PN5 vs. PN59 (Top 25 DEGs)', fontsize=20)
plt.tight_layout()
plt.savefig('final_top25_DEGs.png', bbox_inches='tight', dpi=300,  transparent=True)

plt.show()


In [ ]:
from adjustText import adjust_text

In [ ]:
#subset_genes = pd.DataFrame(subset.uns['rank_genes_groups']['names'])
#subset_genes

In [ ]:
gp.get_library_name()

In [ ]:
upreg_df_new = pd.DataFrame({'SigUpreg': upreg_genes})
downreg_df_new = pd.DataFrame({'SigDownreg': downreg_genes})

In [ ]:
num_clusters = upreg_df_new.shape[1]
background_gene_list = adata.raw.var_names.tolist()
gene_sets = ['PanglaoDB_Augmented_2021', 'GO_Biological_Process_2023', 'GO_Cellular_Component_2023' ]

for cluster_number, cluster_name in enumerate(upreg_df_new.columns):
    if cluster_number >= num_clusters:
        break

    fig, axes = plt.subplots(1, 3, figsize=(40, 10))
    plt.subplots_adjust(wspace=1.5, hspace=2)
    plt.suptitle('PN5 vs. PN59 upregulated genes ($\it{p}$ < 0.05)', fontsize=36, fontweight='bold', y=1.02)

    for gene_set, ax in zip(gene_sets, axes.flatten()):
        gene_list = upreg_df_new[cluster_name].dropna().tolist()

        if not gene_list:
            continue

        enr_results = gp.enrichr(gene_list=gene_list,
                                gene_sets=[gene_set],
                                organism='human',
                                outdir=None,
                                background=background_gene_list)

        enr_results_sorted = enr_results.results.sort_values(by='P-value')

        top_terms = enr_results_sorted.head(10)['Term'].apply(lambda x: x[:50] + '...' if len(x) > 35 else x)
        top_p_values = enr_results_sorted.head(10)['P-value']

        ax.barh(top_terms, -1 * np.log10(top_p_values), color='slategray')
        ax.set_xlabel('-log10(p-value)', fontsize=18)
        ax.set_title(gene_set, fontsize=26)
        ax.tick_params(axis='y', labelsize=26)
        ax.xaxis.grid(False)
        ax.yaxis.grid(False)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_aspect('auto')
        ax.set_ylim([-0.5, 10])

    plt.tight_layout()
    plt.savefig(f'final_{cluster_name}_analysis_PN5vsPN59.png', bbox_inches='tight', dpi=300, transparent=True)
    plt.show()


In [ ]:
num_clusters = downreg_df_new.shape[1]
background_gene_list = adata.raw.var_names.tolist()
gene_sets = ['PanglaoDB_Augmented_2021', 'GO_Biological_Process_2023', 'GO_Cellular_Component_2023' ]

for cluster_number, cluster_name in enumerate(downreg_df_new.columns):
    if cluster_number >= num_clusters:
        break

    fig, axes = plt.subplots(1, 3, figsize=(40, 10))
    plt.subplots_adjust(wspace=1.5, hspace=2)
    plt.suptitle('PN5 vs. PN59 downregulated genes ($\it{p}$ < 0.05)', fontsize=36, fontweight='bold', y=1.02)

    for gene_set, ax in zip(gene_sets, axes.flatten()):
        gene_list = downreg_df_new[cluster_name].dropna().tolist()

        if not gene_list:
            continue

        enr_results = gp.enrichr(gene_list=gene_list,
                                gene_sets=[gene_set],
                                organism='human',
                                outdir=None,
                                background=background_gene_list)

        enr_results_sorted = enr_results.results.sort_values(by='P-value')

        top_terms = enr_results_sorted.head(10)['Term'].apply(lambda x: x[:50] + '...' if len(x) > 35 else x)
        top_p_values = enr_results_sorted.head(10)['P-value']

        ax.barh(top_terms, -1 * np.log10(top_p_values), color='slategray')
        ax.set_xlabel('-log10(p-value)', fontsize=18)
        ax.set_title(gene_set, fontsize=26)
        ax.tick_params(axis='y', labelsize=26)
        ax.xaxis.grid(False)
        ax.yaxis.grid(False)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_aspect('auto')
        ax.set_ylim([-0.5, 10])

    plt.tight_layout()
    plt.savefig(f'final_{cluster_name}_analysis_PN5vsPN59.png', bbox_inches='tight', dpi=300, transparent=True)
    plt.show()


In [ ]:
#selected_genes = subset.raw[:, genes_to_show].X.toarray()
#sample_labels = subset.obs['Sample'].tolist()
#df = pd.DataFrame(selected_genes, columns=genes_to_show)
#df['Sample'] = sample_labels

In [ ]:
#df.set_index('Sample', inplace=True)
#df_zscore = df.apply(zscore)
#gene_list

In [ ]:
color_s2 = {'Hindbrain - Differentiated':'#469990', 
           'Hindbrain - SCs':'#9A6324', 
           'Hindbrain - Serotonergic':'#800000', 
           'Midbrain - Dopaminergic':'#808000', 
           'Motor neurons':'#000075',
           'Neural - SCs':'#e6194B', 
           'Serotonergic':'#f58231', 
           'Spinal - GABAergic':'#ffe119', 
           'Spinal - SCs':'#bfef45', 
           'Spinal - Schwann SCs':'#3cb44b',
           }

legend_handles3 = [Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=label) for label, color in color_s2.items()]
fig, ax = plt.subplots(figsize=(1, 1))
ax.axis('off')
ax.legend(handles=legend_handles3, ncol=1, loc='center', frameon=False)
plt.savefig('legend_celltype.png', bbox_inches='tight', transparent=True, dpi=300)
plt.show()

In [ ]:
color_s2 = ['#469990', 
           '#9A6324', 
           '#800000', 
           '#808000', 
           '#000075',
           '#e6194B', 
           '#f58231', 
           '#ffe119', 
           '#bfef45', 
           '#3cb44b',
          ]

In [ ]:

cluster_names = {
    '0': 'Hindbrain - SCs',
    '1': 'Hindbrain - SCs',
    '2': 'Hindbrain - SCs',
    '3': 'Hindbrain - SCs',
    '4': 'Hindbrain - SCs',
    '5': 'Motor neurons',
    '6': 'Spinal - SCs',
    '7': 'Hindbrain - SCs',
    '8': 'Midbrain - Dopaminergic',
    '9': 'Spinal - SCs',
    '10': 'Spinal - SCs',
    '11': 'Hindbrain - Serotonergic',
    '12': 'Spinal - Schwann SCs',
    '13': 'Spinal - GABAergic',
    '14': 'Neural - SCs',
    '15': 'Serotonergic',
    '16': 'Hindbrain - Differentiated',

}

adata.obs['cell_type'] = adata.obs['leiden'].map(cluster_names)

# UMAP cell_types

sc.set_figure_params(transparent=True, dpi=80, fontsize=14, dpi_save=300, figsize=(10, 8))
sns.set_palette("deep")

plt.figure(figsize=(10, 8))
sc.pl.umap(adata,
           color='cell_type',  
           palette=color_s2,  
           use_raw=False,
           title=[''],
           frameon=False,
           legend_loc=False,
           legend_fontsize=16,
           #legend_fontoutline=1,
           size=30,
           alpha=0.8,
           linewidth=0.5,
           show=False
           )

#plt.title('Samples', fontsize=20)
#plt.xlabel('UMAP1', fontsize=20)
#plt.ylabel('UMAP2', fontsize=20)
#plt.gca().spines['top'].set_visible(False)
#plt.gca().spines['right'].set_visible(False)

ax.set_aspect('equal')
plt.savefig('final__celltypes_NSCs.png', bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
cell_type_counts = adata.obs['cell_type'].value_counts()
print(cell_type_counts)

In [ ]:
cell_type_counts = adata.obs['cell_type'].value_counts()

plt.figure(figsize=(8, 6))
ax = plt.axes()
bars = cell_type_counts.plot(kind='barh', color=color_t, ax=ax)
ax.grid(False) 

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#ax.spines['bottom'].set_visible(False)
#ax.spines['left'].set_visible(False)
plt.xlabel('Number of Cells')

for i, value in enumerate(cell_type_counts):
    plt.text(value + 0.05, i, str(value), ha='left', va='center')
plt.savefig('final_number_of_cells_in_celltype.png', bbox_inches='tight', dpi=300,  transparent=True)
plt.show()


In [ ]:
sc.tl.rank_genes_groups(adata, groupby='Sample', method='wilcoxon', use_raw=True)

In [ ]:
#sns.set_palette("rocket")
fig, ax = plt.subplots(figsize=(0.2, 2))  # Adjust the figsize as needed for your design
bar_lgd = plt.scatter([0, 1], [0, 1], c=[0, 1], cmap="rocket")
cbar = plt.colorbar(bar_lgd, cax=ax, orientation='vertical', ticks=[0, 1])
cbar.set_label('Scaled expression', rotation=270, labelpad=10, fontsize=10)
cbar.ax.tick_params(labelsize=12)
cbar.ax.set_yticklabels(['0', '1'])
ax.set_xticks([])
ax.set_xticklabels([])
plt.savefig('legend_heatmap.png', bbox_inches='tight', transparent=True, dpi=300)
plt.show()



In [ ]:
sc.set_figure_params(transparent=True, fontsize=14, dpi_save=300)
sc.pl.rank_genes_groups_heatmap(adata,  
                                n_genes=10, 
                                groupby='Sample', 
                                cmap='rocket', 
                                dendrogram=True, 
                                figsize=(6, 10),
                                use_raw=True,
                                swap_axes=True,
                                show_gene_labels=True,
                                standard_scale='var',
                                show=False,
                                save='final_top10genes_heatmap.png'                               
                               )

In [ ]:
sc.tl.dendrogram(adata, 'Sample', n_pcs=20)
axes_list = sc.pl.correlation_matrix(adata, 'Sample', show=False)

for ax in axes_list:
    ax.yaxis.grid(False)
    ax.xaxis.grid(False)
#plt.savefig('correlation_matrix.png', bbox_inches='tight', dpi=300, transparent=True)
plt.show()


# markers in clusters


In [ ]:
#finding markers in clusters
sc.tl.rank_genes_groups(adata, groupby='leiden', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False, show=False, ncols=3)
plt.savefig('final_t-test ranked genes leiden.png', bbox_inches='tight', dpi=300,  transparent=True)


In [ ]:
color_s2 = ['#0fbabd', '#efd39b', '#fbbd3c', '#bc131f', '#876c56']

In [ ]:
sc.set_figure_params(transparent=True, dpi=80, fontsize=14, dpi_save=300, figsize=(10, 8))
selected_genes = {
                'Forebrain' :['SIX3', 'FOXG1', 'SP8', 'EOMES'],
                 'Midbrain':['CLSTN2', 'PTPRO','LMX1B', 'FGF8', 'WNT5A', 'EN2'],
                'Hindbrain':['HOXB4', 'HOXB2', 'HOXA2', 'ERBB4', 'GBX2', 'PAX3'],
                'Spinal\ncord':['HOXB9', 'HOXB8', 'HOXB7', 'HOXB6', 'CADPS2'],
                'Seroton-\nergic' :['FEV', 'TPH2', 'SLC6A4','HTR7', 'HTR2A'],
                'GABA-\nergic' :['GAD1', 'GAD2', 'SLC6A1'],
                'Dopamin-\nergic' :['TH', 'SLC6A3'],
                'Neural stem cells':['SOX1', 'SOX2', 'DLL1', 'FABP7', 'HES5', 'NES','PAX6', 'NOTCH1'],
                'Differentiated ':['L1CAM','MAP2', 'TUBB3', 'NEFM', 'NEFL'],
                 'Schwann\ncells' :['MPZ', 'SOX10', 'S100B'],
                'Motor\nneurons' :['LHX3', 'NKX6-1', 'AQP4'],
                }


sc.pl.matrixplot(adata, 
                 selected_genes, 
                 #use_raw = False,
                 groupby='leiden', 
                 dendrogram=True, 
                 cmap='Reds',
                 #swap_axes=True,
               standard_scale='var',
                 var_group_rotation=0,
                colorbar_title='column scaled\nexpression',
                 #save='final_matrixplot.png'
                )

In [ ]:
num_tot_cells = adata.obs.groupby(['Sample']).count()
num_tot_cells = dict(zip(num_tot_cells.index, num_tot_cells.n_genes))
cell_type_counts = adata.obs.groupby(['Sample', 'cell_type']).count()
cell_type_counts = cell_type_counts[cell_type_counts.sum(axis=1)>0].reset_index()
cell_type_counts = cell_type_counts.rename(columns={'Sample': 'sample_column_name'})
cell_type_counts['total_cells'] = cell_type_counts['sample_column_name'].map(num_tot_cells).astype(int)
cell_type_counts['frequency'] = cell_type_counts.leiden/cell_type_counts.total_cells

In [ ]:
leiden_counts = adata.obs['leiden'].value_counts()
total_cells = len(adata.obs)

percentage_cells = (leiden_counts / total_cells) * 100
plt.figure(figsize=(10, 6))
ax = plt.axes()  # Get the current axes
percentage_cells.plot(kind='bar', color='skyblue', ax=ax)
ax.grid(False)  # Turn off the grid
plt.title('Percentage of Cells in Each Cell Type Cluster')
plt.xlabel('leiden cluster')
plt.ylabel('Percentage of Cells')
plt.xticks(rotation=45, ha='right')
plt.savefig('final_percentage_of_cells.png', bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
agg_data = adata.obs.groupby(['cell_type', 'Sample']).size().unstack().fillna(0)
agg_data_normalized = agg_data.div(agg_data.sum(axis=1), axis=0) * 100
colors = [color_s1[sample] for sample in agg_data_normalized.columns]

plt.figure(figsize=(12, 8))
plt.gca().set_facecolor('none') 

for i, sample in enumerate(agg_data_normalized.columns):
    plt.bar(
        agg_data_normalized.index,
        agg_data_normalized[sample],
        label=sample,
        color=colors[i],
        width=0.8,
        bottom=agg_data_normalized.iloc[:, :i].sum(axis=1),
        alpha=0.8
    )

# Customization
plt.ylabel('Percentage of cells', fontsize=20)
plt.legend(bbox_to_anchor=(0.99, 1.024), loc='upper left', fontsize=30)
plt.xticks(rotation=45, ha='right', fontsize=36)
plt.yticks(fontsize=20)
plt.ylim(0, 100)
plt.grid(False)
plt.savefig('final_Cell_Proportion.png', bbox_inches='tight', transparent=True, dpi=300)
plt.show()

In [ ]:
agg_data_normalized 

In [ ]:

my_order = ['6', '9', '10', '2', '12', '3', '7', '4', '0', '1', '14', '5', '16', '15', '11', '8', '13']
#my_order = my_order[::-1]
percentage_cells = percentage_cells.loc[my_order]

fig, ax = plt.subplots(figsize=(2, 6))
bars = ax.barh(percentage_cells.index, percentage_cells, color='black')  
ax.invert_yaxis()
ax.yaxis.tick_right()
ax.yaxis.set_label_position("right")
ax.set_xlim(0, 25)
ax.invert_xaxis()
ax.yaxis.set_label_position("left") 

plt.ylabel('cluster')
plt.xlabel('Percentage of Cells')
plt.grid(False)

for bar in bars:
    plt.text(bar.get_width() +8, bar.get_y() + bar.get_height() / 2, f'{bar.get_width():.1f}%', 
             va='center', ha='left', fontsize=10, color='black')

plt.show()


In [ ]:
gene_index = np.where(adata.raw.var_names == 'HOXB9')[0]
expressing_cells = np.sum(adata.raw.X[:, gene_index] > 0)
print(f"The number of cells expressing SIX3 is: {expressing_cells}")


In [ ]:
cell_genes = [gene for gene_list in selected_genes.values() for gene in gene_list]

# Filter out non-gene entries
gene_names = [gene for gene in cell_genes if isinstance(gene, str)]

# Calculate the number of expressing cells for each gene
expressing_cells_counts = []

for gene in gene_names:
    gene_index = np.where(adata.raw.var_names == gene)[0]
    expressing_cells = np.sum(adata.raw.X[:, gene_index] > 0)
    expressing_cells_counts.append(expressing_cells)

# Create a bar plot
plt.figure(figsize=(22, 1))
bars = plt.bar(gene_names, expressing_cells_counts, color='slategray')
plt.xticks(rotation=90, ha='center')
plt.grid(False)

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.yticks([])
plt.tick_params(axis='y', which='both', left=False)
plt.ylabel('Number of cells', fontsize=16, ha='center', va='center')


# Add text labels on top of each bar
for bar, count in zip(bars, expressing_cells_counts):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 20, str(count),fontsize=14, ha='center', va='bottom', rotation=90)

plt.show()


In [ ]:
sc.set_figure_params(fontsize=20)


fig = plt.figure(figsize=(22, 8))
gs = gridspec.GridSpec(1, 2, width_ratios=[0.1, 1], height_ratios=[0.01])  # 1 row, 2 columns, with different widths


ax = plt.subplot(gs[0])
bars = ax.barh(percentage_cells.index, percentage_cells, color='black')
ax.invert_yaxis()
ax.yaxis.tick_right()
ax.yaxis.set_label_position("right")
ax.set_xlim(0, 20)
ax.invert_xaxis()
plt.xlabel('Percentage of cells')
ax.yaxis.set_label_position("left")
plt.ylabel('Clusters')
plt.grid(False)


for bar in bars:
    plt.text(bar.get_width() + 5, bar.get_y() + bar.get_height() / 2, f'{bar.get_width():.1f}%',
             va='center', ha='left', fontsize=14, color='black')


ax.set_yticks([])
ax.set_yticklabels([])


ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)


ax_matrixplot = plt.subplot(gs[1])


gs.update(right=0.90)  
plt.subplots_adjust(left=0.01, right=0.1, wspace=0.05)
bar_plot_height = 0.705  
ax.set_position([ax.get_position().x0, ax.get_position().y0, ax.get_position().width, bar_plot_height])
bar_plot_position = ax.get_position()
new_bar_plot_position = [bar_plot_position.x0, bar_plot_position.y0 - 0.025, bar_plot_position.width, bar_plot_position.height]
ax.set_position(new_bar_plot_position)
sc.pl.matrixplot(adata, selected_genes, groupby='leiden', dendrogram=True, cmap='Reds',
                 standard_scale='var', var_group_rotation=90, colorbar_title='column scaled\nexpression',
                 ax=ax_matrixplot, show = False)



plt.savefig('final_matrix_with_bar_plot.png', bbox_inches='tight', transparent=True, dpi=300)
plt.show()


In [ ]:
#pie charts
sample_names = adata.obs['Sample'].unique()
color_s1 = {'PN5': 'darkred', 'PN21': 'purple', 'PN41': 'darkorange', 'PN59': 'gold'}

num_genes = len(cell_genes)
fig, axes = plt.subplots(1, num_genes, figsize=(3 * num_genes, 3))


for gene, ax in zip(cell_genes, axes):
    if gene not in adata.raw.var_names:
        print(f"Skipping gene '{gene}' as it is not present in the raw data.")
        continue

    expressing_cells_counts = []

    for sample in sample_names:
        sample_cells = adata.obs['Sample'] == sample
        gene_index = np.where(adata.raw.var_names == gene)[0]
        expressing_cells = np.sum((adata.raw.X[sample_cells, gene_index] > 0).A)
        expressing_cells_counts.append(expressing_cells)

    total_cells_per_sample = np.sum(adata.obs['Sample'].isin(sample_names))

    # Check if the total number of expressing cells is less than 10
    if np.sum(expressing_cells_counts) < 10:
        ax.pie([1], colors=['lightgrey'], autopct='', startangle=90)
        ax.set_title(f'{gene} Expression (< 10 cells)')
        continue

    percentage_expression = [count / total_cells_per_sample * 100 for count in expressing_cells_counts]
    colors = [color_s1.get(sample, 'gray') for sample in sample_names]
    ax.pie(percentage_expression, autopct='', startangle=90, colors=colors)
    ax.set_title(f'{gene}')

plt.tight_layout()
plt.show()


In [ ]:
cell_genes = [gene for gene_list in selected_genes.values() for gene in gene_list]

def plot_gene_pie(ax, gene, adata, color_s1):
    if gene not in adata.raw.var_names:
        print(f"Skipping gene '{gene}' as it is not present in the raw data.")
        return
    expressing_cells_counts = []

    for sample in sample_names:
        sample_cells = adata.obs['Sample'] == sample
        gene_index = np.where(adata.raw.var_names == gene)[0]
        expressing_cells = np.sum((adata.raw.X[sample_cells, gene_index] > 0).A)
        expressing_cells_counts.append(expressing_cells)

    total_cells_per_sample = np.sum(adata.obs['Sample'].isin(sample_names))

    if np.sum(expressing_cells_counts) < 10:
        ax.pie([1], colors=['lightgrey'], autopct='', startangle=90)
        ax.set_title('')
        ax.axis('equal')  
        ax.set_xticks([])  
        ax.set_yticks([])  
        ax.set_facecolor('None')
        return

    percentage_expression = [count / total_cells_per_sample * 100 for count in expressing_cells_counts]

    colors = [color_s1.get(sample, 'gray') for sample in sample_names]

    ax.pie(percentage_expression, autopct='', startangle=90, colors=colors)
    ax.set_title('')
    ax.axis('equal')  
    ax.set_xticks([])  
    ax.set_yticks([])  
    ax.set_facecolor('None')

sample_names = adata.obs['Sample'].unique()
color_s1 = {'PN5': 'darkred', 'PN21': 'purple', 'PN41': 'darkorange', 'PN59': 'gold'}
num_genes = len(cell_genes)
fig, axes = plt.subplots(1, num_genes, figsize=(3 * num_genes, 3))

for i, gene in enumerate(cell_genes):
    ax_pie = axes[i]
    plot_gene_pie(ax_pie, gene, adata, color_s1)

plt.subplots_adjust(wspace=0)
plt.show()


In [ ]:

sc.set_figure_params(fontsize=20)

fig = plt.figure(figsize=(40, 14))
gs = gridspec.GridSpec(2, 2, width_ratios=[0.1, 1], height_ratios=[1, 1])

# Bar plot on the left
ax_left = plt.subplot(gs[1, 0])
bars_left = ax_left.barh(percentage_cells.index, percentage_cells, color='black')
ax_left.invert_yaxis()
ax_left.yaxis.tick_right()
ax_left.yaxis.set_label_position("right")
ax_left.set_xlim(0, 20)
ax_left.invert_xaxis()
ax_left.set_xlabel('Percentage of cells')
ax_left.yaxis.set_label_position("left")
ax_left.set_ylabel('Clusters')
ax_left.grid(False)

for bar in bars_left:
    plt.text(bar.get_width() + 5, bar.get_y() + bar.get_height() / 2, f'{bar.get_width():.1f}%',
             va='center', ha='left', fontsize=14, color='black')

ax_left.set_yticks([])
ax_left.set_yticklabels([])
ax_left.spines['top'].set_visible(False)
ax_left.spines['right'].set_visible(False)
ax_left.spines['bottom'].set_visible(True)
ax_left.spines['left'].set_visible(False)

# Matrix plot in the center
ax_matrixplot = plt.subplot(gs[1, 1])
sc.pl.matrixplot(adata, selected_genes, groupby='leiden', dendrogram=True, cmap='Reds',
                 standard_scale='var', var_group_rotation=90, colorbar_title='column scaled\nexpression',
                 ax=ax_matrixplot, show=False)

# Bar plot on top
ax_top = plt.subplot(gs[0, 1])
bars_top = ax_top.bar(gene_names, expressing_cells_counts, color='slategray')
ax_top.set_xticks([])
ax_top.grid(False)
ax_top.spines['top'].set_visible(False)
ax_top.spines['right'].set_visible(False)
ax_top.spines['bottom'].set_visible(False)
ax_top.spines['left'].set_visible(False)
ax_top.tick_params(axis='y', which='both', left=False, labelleft=False)



for bar, count in zip(bars_top, expressing_cells_counts):
    ax_top.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 20, str(count),
                fontsize=14, ha='center', va='bottom', rotation=90)
    
ax_top.set_ylabel('Number of cells', fontsize=16, ha='right', va='center', rotation=0)


gs.update(right=0.90)
gs.update(bottom=0.1, top=1, wspace=0.05, hspace=0.1)

ax_left_pos = ax_left.get_position()
ax_left.set_position([ax_left_pos.x0 - 0.03, ax_left_pos.y0 - 0.02, ax_left_pos.width * 1.5, ax_left_pos.height*0.94])

ax_top_pos = ax_top.get_position()
ax_top.set_position([ax_top_pos.x0 - 0.028, ax_top_pos.y0 - -0.13, ax_top_pos.width* 0.965, ax_top_pos.height* 0.2])

ax_matrixplot_pos = ax_matrixplot.get_position()
ax_matrixplot.set_position([ax_matrixplot_pos.x0, ax_matrixplot_pos.y0, ax_matrixplot_pos.width, ax_matrixplot_pos.height])

plt.show()



In [ ]:

sc.set_figure_params(fontsize=20)


fig = plt.figure(figsize=(40, 18))
gs = gridspec.GridSpec(3, 2, width_ratios=[0.1, 1], height_ratios=[1, 1, 1])

ax_left = plt.subplot(gs[2, 0])
bars_left = ax_left.barh(percentage_cells.index, percentage_cells, color='black')
ax_left.invert_yaxis()
ax_left.yaxis.tick_right()
ax_left.yaxis.set_label_position("right")
ax_left.set_xlim(0, 20)
ax_left.invert_xaxis()
ax_left.set_xlabel('Percentage of cells')
ax_left.yaxis.set_label_position("left")
ax_left.set_ylabel('Clusters')
ax_left.grid(False)

for bar in bars_left:
    plt.text(bar.get_width() + 5, bar.get_y() + bar.get_height() / 2, f'{bar.get_width():.1f}%',
             va='center', ha='left', fontsize=14, color='black')

ax_left.set_yticks([])
ax_left.set_yticklabels([])
ax_left.spines['top'].set_visible(False)
ax_left.spines['right'].set_visible(False)
ax_left.spines['bottom'].set_visible(True)
ax_left.spines['left'].set_visible(False)
ax_left.set_zorder(2)

ax_matrixplot = plt.subplot(gs[2, 1])
sc.pl.matrixplot(adata, selected_genes, groupby='leiden', dendrogram=True, cmap='Reds',
                 standard_scale='var', var_group_rotation=0, colorbar_title='column scaled\nexpression',
                 ax=ax_matrixplot, show=False)
ax_matrixplot.set_rasterization_zorder(2)


ax_top = plt.subplot(gs[0, 1])
bars_top = ax_top.bar(gene_names, expressing_cells_counts, color='slategray')
ax_top.set_xticks([])
ax_top.grid(False)
ax_top.spines['top'].set_visible(False)
ax_top.spines['right'].set_visible(False)
ax_top.spines['bottom'].set_visible(False)
ax_top.spines['left'].set_visible(False)
ax_top.tick_params(axis='y', which='both', left=False, labelleft=False)



for bar, count in zip(bars_top, expressing_cells_counts):
    ax_top.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 20, str(count),
                fontsize=14, ha='center', va='bottom', rotation=90)
    
ax_top.set_ylabel('Number of cells', fontsize=16, ha='right', va='center', rotation=0)



sample_names = adata.obs['Sample'].unique()
color_s1 = {'PN5': 'darkred', 'PN21': 'purple', 'PN41': 'darkorange', 'PN59': 'gold'}


num_genes = len(cell_genes)
ax_pie = plt.subplot(gs[1, 0:2])

total_width = 0.8  
width_per_pie = total_width / num_genes


for i, gene in enumerate(cell_genes):
    ax_pie_sub = ax_pie.inset_axes([i * width_per_pie, 0, width_per_pie, 1])
    plot_gene_pie(ax_pie_sub, gene, adata, color_s1)
ax_pie.grid(False)
ax_pie.set_frame_on(False) 
ax_pie.set_xticks([])
ax_pie.set_yticks([])
ax_pie.spines['top'].set_visible(False)
ax_pie.spines['right'].set_visible(False)
ax_pie.spines['bottom'].set_visible(False)
ax_pie.spines['left'].set_visible(False)


gs.update(right=0.90)
gs.update(bottom=0.1, top=1, wspace=0.05, hspace=0.1)

ax_left_pos = ax_left.get_position()
ax_left.set_position([ax_left_pos.x0 -0.009, ax_left_pos.y0 - 0.011, ax_left_pos.width * 1.2, ax_left_pos.height*0.94])

ax_top_pos = ax_top.get_position()
ax_top.set_position([ax_top_pos.x0 - 0.0278, ax_top_pos.y0 - 0.28, ax_top_pos.width* 0.965, ax_top_pos.height* 0.2])

ax_matrixplot_pos = ax_matrixplot.get_position()
ax_matrixplot.set_position([ax_matrixplot_pos.x0, ax_matrixplot_pos.y0, ax_matrixplot_pos.width, ax_matrixplot_pos.height])

ax_pie_pos = ax_pie.get_position()
ax_pie.set_position([ax_pie_pos.x0 + 0.081, ax_pie_pos.y0 -0.14, ax_pie_pos.width*0.975, ax_pie_pos.height])


plt.savefig('final_heatmap.png', bbox_inches='tight', transparent=True, dpi=300)
plt.show()



In [ ]:
legend_handles2 = [Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=label) for label, color in color_s1.items()]
legend_handles2.append(Patch(color='lightgrey', label='< 10 cells'))
fig, ax = plt.subplots(figsize=(1, 1))
ax.axis('off')
ax.legend(handles=legend_handles2, ncol=1, loc='center')
plt.savefig('legend_heatmap.png', bbox_inches='tight', transparent=True, dpi=300)
plt.show()
